In [17]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
tf.reset_default_graph()

import os
LOGDIR = os.getcwd() +'/4/'
print(LOGDIR)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
/Users/vinay/Desktop/deep-learning/tensorboard/4/


# layer definition

In [18]:
#define simple convolution layer
def conv_layer(input, channels_in, channels_out, name='convolution'):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([5,5,channels_in, channels_out], stddev=0.1), name='W')
        b = tf.Variable(tf.constant(0.1, shape=[channels_out]), name='B')
        conv = tf.nn.conv2d(input, w, strides=[1,1,1,1], padding='SAME')
        activation = tf.nn.relu(conv + b)
        tf.summary.histogram('weights', w)
        tf.summary.histogram('biases', b)
        tf.summary.histogram('activations', activation)
        return activation

#define a fully connected layer
def fc_layer(input, channels_in, channels_out, name='Fc'):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([channels_in,channels_out], stddev=0.1), name='W')
        b = tf.Variable(tf.constant(0.1, shape=[channels_out]), name='B')
        activation = tf.nn.relu(tf.matmul(input, w)+b)
        tf.summary.histogram('weights', w)
        tf.summary.histogram('biases', b)
        tf.summary.histogram('activations', activation)
        return activation

# Create model

In [19]:

def mnist_model(learning_rate, use_2_conv_layer, use_2_fully_conn_layer, hyper_string):
    tf.reset_default_graph()
    sess = tf.Session()

    #setup placeholders
    x = tf.placeholder(tf.float32, shape=[None,784], name='x')
    y = tf.placeholder(tf.float32, shape=[None,10], name='labels')
    x_image = tf.reshape(x, [-1, 28,28,1])
    tf.summary.image('input_image', x_image, 3)

    #===============================================================
    #feed forward network
    #create network
    if use_2_conv_layer:
        conv1 = conv_layer(x_image, 1, 32, 'conv1')
        pool1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

        conv2 = conv_layer(pool1, 32, 64, 'conv2')
        pool2 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
        flattened = tf.reshape(pool2, [-1, 7*7*64])
    else:
        conv1 = conv_layer(x_image, 1, 64, 'conv')
        pool1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
        pool2 = tf.nn.max_pool(pool1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
        flattened = tf.reshape(pool2, [-1, 7*7*64])

    if use_2_fully_conn_layer:
        fc1 = fc_layer(flattened, 7*7*64, 1024, 'fc1')
        embedding_input = fc1
        embedding_size = 1024
        logits = fc_layer(fc1, 1024, 10, 'fc2')    
    else:
        embedding_input = flattened
        embedding_size = 7*7*64
        logits = fc_layer(flattened, 7*7*64, 10, 'fc')

    #===============================================================
    # loss & training
    #compute cross entropy
    with tf.name_scope('cross_entropy'):
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
        tf.summary.scalar('cross_entropy_scalar', cross_entropy)

    #use adamoptimizer to train network
    with tf.name_scope('train_step'):
        train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

    #compute accuracy
    with tf.name_scope('accuraracy'):
        correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar('accuracy', accuracy)
    
    #===============================================================
    #train the model
    #initialise all varibles
    sess.run(tf.global_variables_initializer())

    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter(LOGDIR + hyper_string)
    writer.add_graph(sess.graph)

    embedding = tf.Variable(tf.zeros([1024, embedding_size]), name="test_embedding")
    assignment = embedding.assign(embedding_input)
    saver = tf.train.Saver()
    
    
    config = tf.contrib.tensorboard.plugins.projector.ProjectorConfig()
    embedding_config = config.embeddings.add()
    embedding_config.tensor_name = embedding.name
    embedding_config.sprite.image_path = LOGDIR + 'sprite_1024.png'
    embedding_config.metadata_path = LOGDIR + 'labels_1024.tsv'
    # Specify the width and height of a single thumbnail.
    embedding_config.sprite.single_image_dim.extend([28, 28])
    tf.contrib.tensorboard.plugins.projector.visualize_embeddings(writer, config)
    
    #train for 1000 steps
    for i in range(1000):
        batch = mnist.train.next_batch(100)

        #occasionally report accuracy
        if i % 200 == 0:
            [train_accuracy] = sess.run([accuracy], feed_dict={x:batch[0], y:batch[1]})
            print('step %d, training accuracy %g' %(i, train_accuracy))

        if i % 5 == 0:
            s = sess.run(merged_summary, feed_dict={x:batch[0], y:batch[1]})
            writer.add_summary(s, i)
            
        if i % 500 == 0:
            sess.run(assignment, feed_dict={x: mnist.test.images[:1024], y: mnist.test.labels[:1024]})
            saver.save(sess, os.path.join(LOGDIR, "model.ckpt"), i)


        #run training step
        sess.run(train_step, feed_dict={x:batch[0],y:batch[1]})

def make_hparam_string(learning_rate, use_two_fc, use_two_conv):
    conv_param = "conv=2" if use_two_conv else "conv=1"
    fc_param = "fc=2" if use_two_fc else "fc=1"
    return "lr_%.0E,%s,%s" % (learning_rate, conv_param, fc_param)

In [20]:
for learning_rate in [1E-4]:
    # Include "False" as a value to try different model architectures
    for use_two_fc in [True]:
        for use_two_conv in [True]:
            # Construct a hyperparameter string for each one (example: "lr_1E-3,fc=2,conv=2)
            hparam = make_hparam_string(learning_rate, use_two_fc, use_two_conv)
            print('Starting run for %s' % hparam)

            # Actually run with the new settings
            mnist_model(learning_rate, use_two_fc, use_two_conv, hparam)


Starting run for lr_1E-04,conv=2,fc=2
step 0, training accuracy 0.15
step 200, training accuracy 0.09
step 400, training accuracy 0.07
step 600, training accuracy 0.05
step 800, training accuracy 0.13
